# 3D Tomographic Reconstruction from 2D Flat Panel Detector Images

This notebook demonstrates the complete workflow for reconstructing 3D volumetric data from a series of 2D projections obtained from a flat panel detector, with support for reading metadata from text files.

## Overview
1. Data loading and metadata extraction
2. Preprocessing and filtering
3. Reconstruction algorithm implementation
4. 3D visualization
5. Export to 3D file formats

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.ndimage as ndimage
from skimage import io, transform, filters
import os
import sys
from tqdm import tqdm
import plotly.graph_objects as go

# Add the src directory to the path
sys.path.append('../src')

# Import our custom modules
from preprocessing import preprocess_projections
from reconstruction import filtered_backprojection, art_reconstruction, sirt_reconstruction, fdk_reconstruction
from visualization import show_projections, show_sinogram, show_volume_slices, show_volume_3slice, render_surface_plotly
from utils import load_projections_with_metadata, create_projection_geometry, save_numpy_as_vtk, save_volume_as_tiff_stack

## 1. Data Loading and Metadata Extraction

In this section, we load the 2D projections and associated metadata from text files. The system can automatically find and parse metadata files that accompany TIFF images.

In [ ]:
# Define paths
data_path = '../data/raw/'
processed_path = '../data/processed/'
reconstructions_path = '../data/reconstructions/'

# Create directories if they don't exist
for path in [processed_path, reconstructions_path]:
    if not os.path.exists(path):
        os.makedirs(path)

# Parameters for loading data
file_pattern = '*.tif*'  # Pattern for TIFF files
angle_pattern = '_([\d.]+)deg'  # Pattern to extract angles from filenames (as fallback)

# Check if data exists
if os.path.exists(data_path) and len(os.listdir(data_path)) > 0:
    # Load projections with metadata
    print("Loading projections and metadata from files...")
    projections, angles, metadata = load_projections_with_metadata(data_path, file_pattern, angle_pattern)
    
    print(f"Loaded {len(projections)} projections with shape {projections[0].shape}")
    print(f"Angle range: {angles.min():.2f}° to {angles.max():.2f}°")
    
    # Print available metadata
    print("\nMetadata found:")
    for key, value in metadata.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for subkey, subvalue in value.items():
                print(f"    {subkey}: {subvalue}")
        else:
            print(f"  {key}: {value}")
    
    # Display sample projections
    indices = [0, len(projections)//3, 2*len(projections)//3, len(projections)-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)
else:
    print("No data found in the raw data directory. Please add TIFF files before proceeding.")
    print("Creating a simulation phantom for demonstration...")
    
    # Create a simple phantom (a cube with a sphere inside)
    phantom_size = 128
    phantom = np.zeros((phantom_size, phantom_size, phantom_size))
    
    # Add a cube
    margin = 20
    phantom[margin:-margin, margin:-margin, margin:-margin] = 0.5
    
    # Add a sphere
    x, y, z = np.ogrid[:phantom_size, :phantom_size, :phantom_size]
    center = phantom_size // 2
    radius = phantom_size // 4
    sphere = (x - center)**2 + (y - center)**2 + (z - center)**2 <= radius**2
    phantom[sphere] = 1.0
    
    # Create projection angles
    n_angles = 60
    angles = np.linspace(0, 360, n_angles, endpoint=False)
    
    # Create projections
    projections = np.zeros((n_angles, phantom_size, phantom_size))
    
    for i, angle in enumerate(angles):
        # Simple parallel beam projection (sum along rotated z-axis)
        rotated = ndimage.rotate(phantom, angle, axes=(0, 1), reshape=False, order=1)
        projections[i] = np.sum(rotated, axis=0)
    
    # Normalize projections to [0, 1]
    projections = (projections - projections.min()) / (projections.max() - projections.min())
    
    # Create sample metadata
    metadata = {
        'source_detector_distance': 1000.0,
        'source_object_distance': 500.0,
        'exposure_time': 100.0,
        'geometry': {
            'source_origin_dist': 500.0,
            'origin_detector_dist': 500.0
        }
    }
    
    # Save the phantom and projections
    np.save(f"{reconstructions_path}/phantom.npy", phantom)
    np.save(f"{processed_path}/simulated_projections.npy", projections)
    
    # Display sample projections
    indices = [0, n_angles//3, 2*n_angles//3, n_angles-1]
    show_projections(projections, indices)
    
    # Display a sinogram
    show_sinogram(projections)

### Creating Test Metadata Files

This cell can be used to create test metadata files if you want to test the metadata reading functionality.

In [ ]:
def create_test_metadata_files(data_path, tiff_files=None):
    """Create test metadata files for the TIFF files in the given directory."""
    import glob
    import json
    
    # Create directory if it doesn't exist
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    # Get TIFF files if not provided
    if tiff_files is None:
        tiff_files = sorted(glob.glob(os.path.join(data_path, '*.tif*')))
    
    if not tiff_files:
        print("No TIFF files found in the directory.")
        return
    
    # Create directory-level metadata file
    dir_metadata = {
        'source_detector_distance': 1000.0,
        'source_object_distance': 500.0,
        'pixel_size': 0.2,  # mm
        'exposure_time': 100.0,  # ms
        'voltage': 120.0,  # kV
        'current': 50.0  # mA
    }
    
    with open(os.path.join(data_path, 'metadata.txt'), 'w') as f:
        for key, value in dir_metadata.items():
            f.write(f"{key} = {value}\n")
    
    print(f"Created directory metadata file at {os.path.join(data_path, 'metadata.txt')}")
    
    # Create individual metadata files for each TIFF
    n_files = len(tiff_files)
    angles = np.linspace(0, 360, n_files, endpoint=False)
    
    for i, tiff_file in enumerate(tiff_files):
        base_path = os.path.splitext(tiff_file)[0]
        
        # Create metadata file with angle information
        file_metadata = {
            'angle': float(angles[i]),
            'projection_number': i,
            'timestamp': f"2025-05-04 {i//60:02d}:{i%60:02d}:00"
        }
        
        # Save as JSON for even indices, text for odd indices (to test both formats)
        if i % 2 == 0:
            with open(f"{base_path}.json", 'w') as f:
                json.dump(file_metadata, f, indent=2)
        else:
            with open(f"{base_path}.txt", 'w') as f:
                for key, value in file_metadata.items():
                    f.write(f"{key} = {value}\n")
    
    print(f"Created individual metadata files for {n_files} TIFF files")

# Uncomment the following line to create test metadata files
# create_test_metadata_files(data_path)

In [ ]:
# Preprocess the projections
processed_projections = preprocess_projections(
    projections,
    angles=angles,
    normalize=True,
    denoise=True,
    remove_rings=True,
    correct_rotation=True,
    log_transform=True
)

# Display preprocessed projections
show_projections(processed_projections, indices)

## 2. Geometric Setup from Metadata

Define the geometry of our imaging setup using the metadata.

In [ ]:
# Use geometry information from metadata if available
if 'geometry' in metadata:
    source_origin_dist = metadata['geometry']['source_origin_dist']
    origin_detector_dist = metadata['geometry']['origin_detector_dist']
    print(f"Using geometry from metadata:")
else:
    # Default values if not in metadata
    source_origin_dist = 500.0  # mm (distance from source to rotation center)
    origin_detector_dist = 500.0  # mm (distance from rotation center to detector)
    print(f"Using default geometry (metadata not available):")

print(f"  Source to rotation center distance: {source_origin_dist} mm")
print(f"  Rotation center to detector distance: {origin_detector_dist} mm")

detector_shape = projections.shape[1:3]  # (height, width) of detector

# Setup projection geometry
geometry = create_projection_geometry(
    angles,
    detector_shape,
    source_origin_dist,
    origin_detector_dist
)

# Define the size of the reconstruction volume
volume_size = (detector_shape[0], detector_shape[0], detector_shape[0])  # cubic volume
print(f"Reconstruction volume size: {volume_size}")

## 3. Reconstruction Algorithm Implementation

Apply different reconstruction algorithms to create a 3D volume from 2D projections.

In [ ]:
# Filtered Back Projection (FBP) reconstruction
print("Starting Filtered Back Projection reconstruction...")
fbp_volume = filtered_backprojection(
    processed_projections,
    angles,
    volume_size
)
print("FBP reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fbp_volume.npy", fbp_volume)
save_numpy_as_vtk(fbp_volume, f"{reconstructions_path}/fbp_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fbp_volume)

In [ ]:
# Algebraic Reconstruction Technique (ART)
print("Starting ART reconstruction...")
art_volume = art_reconstruction(
    processed_projections,
    angles,
    volume_size,
    iterations=5  # ART is iterative, so we specify the number of iterations
)
print("ART reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/art_volume.npy", art_volume)
save_numpy_as_vtk(art_volume, f"{reconstructions_path}/art_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(art_volume)

In [ ]:
# For cone-beam CT, use FDK algorithm with geometry information from metadata
print("Starting FDK reconstruction...")
fdk_volume = fdk_reconstruction(
    processed_projections,
    geometry,
    volume_size
)
print("FDK reconstruction completed.")

# Save the reconstructed volume
np.save(f"{reconstructions_path}/fdk_volume.npy", fdk_volume)
save_numpy_as_vtk(fdk_volume, f"{reconstructions_path}/fdk_volume.vti")

# Display orthogonal slices of the reconstructed volume
show_volume_3slice(fdk_volume)

## 4. 3D Visualization and Export

Visualize and export the 3D reconstruction results for use in external 3D software.

In [ ]:
# Create interactive 3D visualization with Plotly
# Adjust threshold as needed (higher threshold for sparser result)
try:
    threshold = 0.5 * fbp_volume.max()
    fig = render_surface_plotly(fbp_volume, threshold=threshold, opacity=0.7)
    fig.write_html(f"{reconstructions_path}/fbp_isosurface.html")
    fig.show()
except Exception as e:
    print(f"Error creating 3D visualization: {e}")

In [ ]:
# Export as TIFF stack for compatibility with other 3D software
save_volume_as_tiff_stack(fbp_volume, f"{reconstructions_path}/fbp_slices", "slice")
print(f"Saved TIFF stack to {reconstructions_path}/fbp_slices/")

# Also save as VTK file with metadata-based pixel spacing (if available)
pixel_size = metadata.get('pixel_size', 1.0) if isinstance(metadata, dict) else 1.0
spacing = (pixel_size, pixel_size, pixel_size)
save_numpy_as_vtk(fbp_volume, f"{reconstructions_path}/fbp_volume_scaled.vti", spacing=spacing)
print(f"Saved scaled VTK file with pixel size = {pixel_size} mm")

## 5. Comparing Reconstruction Methods

Compare the quality of different reconstruction algorithms.

In [ ]:
# Compare middle slices of different reconstruction methods
methods = ['FBP', 'ART', 'FDK']
volumes = [fbp_volume, art_volume, fdk_volume]

# Get middle slice
mid_z = volume_size[2] // 2

plt.figure(figsize=(15, 5))
for i, (method, vol) in enumerate(zip(methods, volumes)):
    plt.subplot(1, 3, i+1)
    plt.imshow(vol[:, :, mid_z], cmap='gray')
    plt.title(f"{method} Reconstruction")
    plt.colorbar()
plt.tight_layout()
plt.savefig(f"{reconstructions_path}/method_comparison.png", dpi=300)
plt.show()

## 6. Conclusion

In this notebook, we've demonstrated how to load and process 2D projection images with their associated metadata from text files, and perform 3D tomographic reconstruction. The key advantages of this approach are:

1. Automatic extraction of geometric parameters from metadata files
2. Support for various metadata file formats (JSON, key-value pairs, etc.)
3. Fallback to filename-based angle extraction when metadata is unavailable

The output includes files that can be opened with common 3D visualization software:
- VTK files (.vti) for ParaView, VTK.js, etc.
- TIFF stacks for 3D Slicer, ImageJ, etc.
- NumPy arrays (.npy) for further processing in Python

This approach is particularly useful for working with data from different tomographic imaging systems, where the metadata format may vary.